In [73]:
import pandas as pd

# read the data

X_train = pd.read_csv('pcaX_train.csv').values
y_train = pd.read_csv('pcaY_train.csv').values.ravel().astype(int)
X_valid = pd.read_csv('pcaX_test.csv').values
y_valid = pd.read_csv('pcaY_test.csv').values.ravel().astype(int)

In [74]:
from sklearn.ensemble import RandomForestClassifier

forest_best = RandomForestClassifier(n_estimators=100,
                                     criterion="entropy",
                                     random_state=123)

forest_best.fit(X_train, y_train)

print(f"Training Accuracy: {forest_best.score(X_train, y_train)*100:0.3f}%")
print(f"Validation Accuracy: {forest_best.score(X_valid, y_valid)*100:0.3f}%")


%timeit forest_best.fit(X_train, y_train) #training(fitting) time
%timeit forest_best.score(X_valid, y_valid) #test time

Training Accuracy: 100.000%
Validation Accuracy: 78.425%
4.39 s ± 81.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
110 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
